In [ ]:
import cv2
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
from approach.ResEmoteNet import ResEmoteNet
from ultralytics import YOLO

# Settings for text
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.7
font_color = (0, 0, 255)  # BGR format
thickness = 2
line_type = cv2.LINE_AA

# Emotions labels
emotions = ['happy', 'surprise', 'sad', 'anger', 'disgust', 'fear', 'neutral']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the YOLOv8 model for face detection
face_detecter = YOLO("/root/Project/class/emotion_detection/yolov11m-face.pt")
face_detecter = face_detecter.to(device)

# Load the emotion classification model
emotion_classifier = ResEmoteNet().to(device)
checkpoint = torch.load('best_model.pth', weights_only=True)
emotion_classifier.load_state_dict(checkpoint['model_state_dict'])
emotion_classifier.eval()

# Image transformation pipeline
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Emotion detection function
def detect_emotion(image):
    img_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = emotion_classifier(img_tensor)
        probabilities = F.softmax(outputs, dim=1)
    scores = probabilities.cpu().numpy().flatten()
    rounded_scores = [round(score, 2) for score in scores]
    return rounded_scores

# Get the emotion with the highest probability
def get_max_emotion(x, y, w, h, image):
    crop_img = image[y:y + h, x:x + w]
    pil_crop_img = Image.fromarray(crop_img)
    rounded_scores = detect_emotion(pil_crop_img)
    max_index = np.argmax(rounded_scores)
    max_emotion = emotions[max_index]
    return max_emotion

# Display the max emotion on the image
def print_max_emotion(x, y, image, max_emotion):
    org = (x, y - 15)
    cv2.putText(image, max_emotion, org, font, font_scale, font_color, thickness, line_type)

# Display all emotion scores on the image
def print_all_emotion(x, y, w, h, image):
    crop_img = image[y:y + h, x:x + w]
    pil_crop_img = Image.fromarray(crop_img)
    rounded_scores = detect_emotion(pil_crop_img)
    org = (x + w + 10, y - 20)
    for index, value in enumerate(emotions):
        emotion_str = f'{value}: {rounded_scores[index]:.2f}'
        y = org[1] + 30
        org = (org[0], y)
        cv2.putText(image, emotion_str, org, font, font_scale, font_color, thickness, line_type)

# Detect faces and emotions using YOLO
def detect_bounding_box(image):
    # YOLO face detection
    results = face_detecter(image)
    faces = []
    
    # Extract bounding boxes from YOLO results
    for result in results:
        for box in result.boxes.xyxy:  # Bounding box coordinates
            x1, y1, x2, y2 = box[:4].int().cpu().numpy()
            w, h = x2 - x1, y2 - y1
            faces.append((x1, y1, w, h))
    
    # Process each detected face
    for (x, y, w, h) in faces:
        # Draw bounding box
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # Emotion detection and display
        max_emotion = get_max_emotion(x, y, w, h, image)
        print_max_emotion(x, y, image, max_emotion)
        print_all_emotion(x, y, w, h, image)
    
    return faces

# Open webcam for real-time emotion detection
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect faces and emotions in the frame
    detect_bounding_box(frame)

    # Show the processed frame
    cv2.imshow('Emotion Detection - Webcam', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
